In [19]:
'''
Step2a. Create DF
'''
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import os

# --- Config ---
SMALL_TRADE_THRESHOLD = 0.01  # Min ETH trade size to process

def load_binance_data(file_pattern, is_orderbook=False):
    """Load Binance data from CSV files with pattern matching"""
    files = glob.glob(file_pattern)
    dfs = []
    for file in files:
        try:
            df = pd.read_csv(file)
            # Extract timestamp from filename (format: ETHUSDT_yyyymmdd_hhmm_type.csv)
            parts = os.path.basename(file).split('_')
            timestamp_str = parts[1] + '_' + parts[2]
            timestamp = datetime.strptime(timestamp_str, '%Y%m%d_%H%M')
            df['timestamp'] = timestamp
            if is_orderbook:
                # Add order book type (bids/asks) from filename
                ob_type = parts[3].split('.')[0]  # gets 'bids' or 'asks'
                df['type'] = ob_type
            dfs.append(df)
        except Exception as e:
            print(f"Error loading {file}: {str(e)}")
            continue
    if not dfs:
        return pd.DataFrame()
    combined_df = pd.concat(dfs).sort_values('timestamp')
    # For order book data, ensure numeric columns
    if is_orderbook:
        combined_df['price'] = pd.to_numeric(combined_df['price'], errors='coerce')
        combined_df['amount'] = pd.to_numeric(combined_df['amount'], errors='coerce')
        combined_df.dropna(subset=['price', 'amount'], inplace=True)
    return combined_df

def enforce_nanosecond_precision(df):
    """Convert all datetime columns to datetime64[ns, UTC]"""
    if df is None:
        return df
    df = df.copy()
    datetime_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    for col in datetime_cols:
        # Ensure UTC (handles both naive and aware)
        if not pd.api.types.is_datetime64tz_dtype(df[col]):
            df[col] = pd.to_datetime(df[col]).dt.tz_localize('UTC')
        else:
            df[col] = pd.to_datetime(df[col]).dt.tz_convert('UTC')
        # Force nanosecond precision
        df[col] = pd.to_datetime(df[col], utc=True).astype('datetime64[ns, UTC]')
    return df

def get_eth_trade_size(trade):
    """Extract ETH and USDT amounts from trade row, handles both DataFrame row and dict input."""
    # Allow both DataFrame row and dict
    trade_type = trade['trade_type'].upper()
    # Accept both 'sell' and 'WETH_SELL'
    if trade_type in ['WETH_SELL', 'SELL']:
        return {
            'eth_amount': trade['sold_amount'],
            'usdt_amount': trade['bought_amount'],
            'price_per_eth': trade['price_per_eth'],
            'direction': 'sell'}
    else:  # WETH_BUY or BUY
        return {
            'eth_amount': trade['bought_amount'],
            'usdt_amount': trade['sold_amount'],
            'price_per_eth': trade['price_per_eth'],
            'direction': 'buy'}

if __name__ == "__main__":
    # Load 1inch trade data
    oneinch_df = pd.read_csv('1inch/oneinch_weth_usdt_trades.csv')
    oneinch_df['block_time'] = pd.to_datetime(oneinch_df['block_time'])

    # Load Binance data
    ohlcv_df = load_binance_data('Binance/ETHUSDT_*_ohlcv.csv')
    bids_df = load_binance_data('Binance/ETHUSDT_*_bids.csv', is_orderbook=True)
    asks_df = load_binance_data('Binance/ETHUSDT_*_asks.csv', is_orderbook=True)

    # Combine bids/asks into single orderbook DataFrame
    orderbook_df = pd.concat([bids_df, asks_df]).sort_values(['timestamp', 'type', 'price'])
    orderbook_df = orderbook_df.groupby(['timestamp', 'type']).apply(
        lambda x: x.sort_values('price', ascending=(x.name[1] == 'asks'))
    ).reset_index(drop=True)
    orderbook_df['cum_amount'] = orderbook_df.groupby(['timestamp', 'type'])['amount'].cumsum()

    # Enforce consistent datetime precision
    oneinch_df = enforce_nanosecond_precision(oneinch_df)
    ohlcv_df = enforce_nanosecond_precision(ohlcv_df)
    orderbook_df = enforce_nanosecond_precision(orderbook_df)

    # Create minute-level timestamps for alignment
    oneinch_df['minute'] = oneinch_df['block_time'].dt.floor('min')
    ohlcv_df['minute'] = ohlcv_df['timestamp'].dt.floor('min')
    orderbook_df['minute'] = orderbook_df['timestamp'].dt.floor('min')

    # --- Trade Filtering and Analysis ---
    skipped_trades = {
        'no_orderbook': {'count': 0, 'examples': []},
        'small_size': {'count': 0, 'examples': []},
        'time_alignment': {'count': 0, 'examples': []}}

    # Pre-calc time boundaries
    ob_min_time = orderbook_df['minute'].min()
    ob_max_time = orderbook_df['minute'].max()

    # Analyze trades that will be skipped (for reporting)
    for idx, trade in oneinch_df.iterrows():
        trade_details = get_eth_trade_size(trade)
        eth_amount = trade_details['eth_amount']
        trade_info = {
            'time': trade['block_time'],
            'size': eth_amount,
            'pool': trade.get('pool_name', 'unknown'),
            'tx_hash': trade['tx_hash']}
        trade_minute = trade['minute']

        # Check time alignment
        if not (ob_min_time <= trade_minute <= ob_max_time):
            skipped_trades['time_alignment']['count'] += 1
            skipped_trades['time_alignment']['examples'].append(trade_info)
            continue

        # Check order book exists
        ob_data = orderbook_df[orderbook_df['minute'] == trade_minute]
        if ob_data.empty:
            skipped_trades['no_orderbook']['count'] += 1
            skipped_trades['no_orderbook']['examples'].append(trade_info)
            continue

        # Check trade size
        if eth_amount < SMALL_TRADE_THRESHOLD:
            skipped_trades['small_size']['count'] += 1
            skipped_trades['small_size']['examples'].append(trade_info)

    # --- Data Verification & Reporting ---
    print("\n=== Data Verification ===")
    print("Post-correction datetime types:")
    print("1inch:", oneinch_df['block_time'].dtype)
    print("OHLCV:", ohlcv_df['timestamp'].dtype)
    print("Orderbook:", orderbook_df['timestamp'].dtype)

    print("\n1inch data sample:")
    print("1inch head:\n", oneinch_df[['block_time', 'trade_type', 'sold_amount', 'bought_amount', 'price_per_eth']].head(3))
    print("1inch tail:\n", oneinch_df[['block_time', 'trade_type', 'sold_amount', 'bought_amount', 'price_per_eth']].tail(3))
    print("\nBinance OHLCV head:\n", ohlcv_df.head(3))
    print("\nBinance OHLCV tail:\n", ohlcv_df.tail(3))
    print("\nBinance Order book bids head:\n", orderbook_df[orderbook_df['type'] == 'bids'].head(3))
    print("\nBinance Order book bids tail:\n", orderbook_df[orderbook_df['type'] == 'bids'].tail(3))

    print("\nData ranges:")
    print("1inch range:", oneinch_df['block_time'].min(), "to", oneinch_df['block_time'].max())
    print("Binance OHLCV range:", ohlcv_df['timestamp'].min(), "to", ohlcv_df['timestamp'].max())

    print("\nRecord counts:")
    print(f"1inch trades: {len(oneinch_df)}")
    print(f"Binance OHLCV: {len(ohlcv_df)}")
    print(f"Binance Order Book entries: {len(orderbook_df)}")

    print("\n=== Trade Filtering Report ===")
    print(f"Total trades: {len(oneinch_df)}")
    print(f"Trades that will be skipped: {sum(v['count'] for v in skipped_trades.values())}")

    print("\nSKIP REASONS:")
    for reason, data in skipped_trades.items():
        print(f"\n{reason.replace('_', ' ').title()}: {data['count']}")
        if data['examples']:
            if reason in ['time_alignment', 'no_orderbook']:
                example = data['examples'][-1]
                print(f"First example - Time: {example['time']}, Size: {example['size']} ETH")
            else:
                example = data['examples'][0]
                print(f"Last Example - Time: {example['time']}, Size: {example['size']} ETH")

    # Save preprocessed data for downstream steps
    os.makedirs('processed_data', exist_ok=True)
    oneinch_df.to_pickle('processed_data/oneinch_df.pkl')
    ohlcv_df.to_pickle('processed_data/ohlcv_df.pkl')
    orderbook_df.to_pickle('processed_data/orderbook_df.pkl')

    print("\nPreprocessed .pkl data saved to 'processed_data' dir for Step2b & 2c")

/var/folders/81/tfp9tldx3nq9004lhg26p4zc0000gn/T/ipykernel_95515/3549048568.py:89: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  orderbook_df = orderbook_df.groupby(['timestamp', 'type']).apply(
/var/folders/81/tfp9tldx3nq9004lhg26p4zc0000gn/T/ipykernel_95515/3549048568.py:51: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not pd.api.types.is_datetime64tz_dtype(df[col]):
/var/folders/81/tfp9tldx3nq9004lhg26p4zc0000gn/T/ipykernel_95515/3549048568.py:51: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDty


=== Data Verification ===
Post-correction datetime types:
1inch: datetime64[ns, UTC]
OHLCV: datetime64[ns, UTC]
Orderbook: datetime64[ns, UTC]

1inch data sample:
1inch head:
                  block_time trade_type  sold_amount  bought_amount  \
0 2025-05-07 09:08:47+00:00  WETH_SELL     0.144511     265.918139   
1 2025-05-07 09:08:23+00:00  WETH_SELL    21.539799   39654.090969   
2 2025-05-07 09:07:59+00:00   WETH_BUY   250.000000       0.135428   

   price_per_eth  
0    1840.117926  
1    1840.968498  
2    1845.998845  
1inch tail:
                     block_time trade_type  sold_amount  bought_amount  \
1792 2025-05-05 08:11:23+00:00  WETH_SELL        0.060     109.496378   
1793 2025-05-05 08:10:11+00:00   WETH_BUY       30.000       0.017653   
1794 2025-05-05 08:04:23+00:00  WETH_SELL        0.027      49.488563   

      price_per_eth  
1792    1824.939633  
1793    1699.441320  
1794    1832.909741  

Binance OHLCV head:
                   timestamp     open     high     

In [20]:
'''
Step2bi - Flow Internalization Analysis
'''
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt

# ====================== CONFIG ======================
BINANCE_TAKER_FEE = 0.0002             # 0.02% taker fee
BINANCE_MAKER_FEE = 0.0000             # 0.00% maker fee
SMALL_TRADE_THRESHOLD = 0.01           # Min ETH trade size to analyze
SLIPPAGE_MODEL = lambda x: 0.0002 * x  # 0.0002 Slippage as % of trade size
LOOKFORWARD_WINDOW = 120               # Minutes (fwd) for warehouse strat analysis
CORRELATION_WINDOW = 2.5               # Minutes before/after 1nch trade for corr analysis 

def get_eth_trade_size(trade):
    """Extract all necessary details directly from the trade record"""
    trade_type = str(trade['trade_type']).upper()
    if trade_type in ['WETH_SELL', 'SELL']:
        return {
            'eth_amount': trade['sold_amount'],
            'usdt_amount': trade['bought_amount'],
            'price_per_eth': trade['price_per_eth'],
            'direction': 'sell'}
    else:  # WETH_BUY or BUY
        return {
            'eth_amount': trade['bought_amount'],
            'usdt_amount': trade['sold_amount'],
            'price_per_eth': trade['price_per_eth'],
            'direction': 'buy'}

# ====================== CORE FUNCTIONS ======================
def calculate_hedging_pnl(trade, orderbook_snapshot):
    """Calc PNL from immediate hedging on Binance"""
    try:
        trade_details = get_eth_trade_size(trade)
        eth_amount = trade_details['eth_amount']

        if trade_details['direction'] == 'sell':
            # Hedge by SELLING ETH on Binance, ie. hit highest bid
            execution_price = orderbook_snapshot[orderbook_snapshot['type'] == 'bids']['price'].max()
            slippage = SLIPPAGE_MODEL(eth_amount)
            executed_price = execution_price * (1 - slippage)
            # PnL: Hedge Price - 1inch Price
            pnl = (executed_price - trade_details['price_per_eth']) * eth_amount
        else:  # buy
            # Hedge by BUYING ETH on Binance, i.e., lift the lowest ask
            execution_price = orderbook_snapshot[orderbook_snapshot['type'] == 'asks']['price'].min()
            slippage = SLIPPAGE_MODEL(eth_amount)
            executed_price = execution_price * (1 + slippage)
            # PnL: 1inch Price - Hedge Price
            pnl = (trade_details['price_per_eth'] - executed_price) * eth_amount

        # Subtract fee (always a cost)
        fee = BINANCE_TAKER_FEE
        pnl -= (executed_price * eth_amount * fee)

        return pnl, executed_price
    except Exception as e:
        print(f"Error in hedging calc for trade {trade.get('tx_hash', 'unknown')}: {str(e)}")
        return np.nan, np.nan

def calculate_warehousing_pnl(trade, ohlcv_df, lookforward_window=LOOKFORWARD_WINDOW):
    """Calc PNL from warehousing risk"""
    try:
        trade_details = get_eth_trade_size(trade)
        eth_amount = trade_details['eth_amount']
        trade_time = trade['block_time']
        end_time = trade_time + timedelta(minutes=lookforward_window)
        
        historical = ohlcv_df[
            (ohlcv_df['timestamp'] >= trade_time) & 
            (ohlcv_df['timestamp'] <= end_time)].copy()
        
        if historical.empty:
            return np.nan, np.nan, np.nan, np.nan
        
        # VWAP calc
        total_volume = historical['volume'].sum()
        if total_volume > 0:
            warehouse_price = (historical['close'] * historical['volume']).sum() / total_volume
        else:
            warehouse_price = historical['close'].mean()

        # PnL Calc
        if trade_details['direction'] == 'sell':
            pnl = (warehouse_price - trade_details['price_per_eth']) * eth_amount
        else:  # buy
            pnl = (trade_details['price_per_eth'] - warehouse_price) * eth_amount
            
        return pnl, warehouse_price, np.nan, np.nan  
    except Exception as e:
        print(f"Error in warehouse calc for trade {trade.get('tx_hash', 'unknown')}: {str(e)}")
        return np.nan, np.nan, np.nan, np.nan

def calculate_correlations(trade, oneinch_df, ohlcv_df, correlation_window):
    """Calc px & volume corrs between 1inch and Binance with improved handling"""
    try:
        window = pd.Timedelta(minutes=correlation_window)
        start_time = trade['block_time'] - window
        end_time = trade['block_time'] + window
        
        # Get Binance data in window
        binance_data = ohlcv_df[
            (ohlcv_df['timestamp'] >= start_time) & 
            (ohlcv_df['timestamp'] <= end_time)]
        
        # Get 1inch trades in same window (excl current trade)
        oneinch_data = oneinch_df[
            (oneinch_df['block_time'] >= start_time) & 
            (oneinch_df['block_time'] <= end_time) & 
            (oneinch_df['tx_hash'] != trade['tx_hash'])]
        
        # Initialize results as NaN
        price_corr = volume_corr = np.nan
        min_points = 3  # Minimum data pts required for correlation
        
        # Price Corr
        if len(binance_data) >= min_points and len(oneinch_data) >= min_points:
            binance_px = binance_data.set_index('timestamp')['close'].resample('1min').last().ffill()
            oneinch_px = oneinch_data.set_index('block_time')['price_per_eth'].resample('1min').last().ffill()
            aligned_px = pd.concat([binance_px.rename('binance'), oneinch_px.rename('oneinch')], axis=1).dropna()
            if len(aligned_px) >= min_points:
                with np.errstate(divide='ignore', invalid='ignore'):
                    price_corr = aligned_px['binance'].corr(aligned_px['oneinch'])
        
        # Volume Corr
        if len(binance_data) >= min_points and len(oneinch_data) >= min_points:
            try:
                binance_vol = binance_data.set_index('timestamp')['volume'].resample('1min').sum()
                oneinch_vol = oneinch_data.apply(get_eth_trade_size, axis=1).apply(lambda x: x['eth_amount']).groupby(oneinch_data['block_time'].dt.floor('min')).sum()
                binance_vol_norm = (binance_vol - binance_vol.mean()) / (binance_vol.std() if binance_vol.std() != 0 else 1)
                oneinch_vol_norm = (oneinch_vol - oneinch_vol.mean()) / (oneinch_vol.std() if oneinch_vol.std() != 0 else 1)
                vol_aligned = pd.concat([binance_vol_norm.rename('binance_vol'), oneinch_vol_norm.rename('oneinch_vol')], axis=1).dropna()
                if len(vol_aligned) >= min_points:
                    with np.errstate(divide='ignore', invalid='ignore'):
                        volume_corr = vol_aligned['binance_vol'].corr(vol_aligned['oneinch_vol'])
            except Exception:
                pass
        
        return (
            round(price_corr, 4) if not np.isnan(price_corr) else np.nan,
            round(volume_corr, 4) if not np.isnan(volume_corr) else np.nan)
    except Exception as e:
        print(f"Correlation error for trade {trade.get('tx_hash', 'unknown')}: {str(e)}")
        return np.nan, np.nan

def flow_internalization_model(oneinch_df, ohlcv_df, orderbook_df):
    """Run the flow internalization analysis with complete output"""
    results = []
    ob_min_time = orderbook_df['minute'].min()
    ob_max_time = orderbook_df['minute'].max()
    ob_times = set(orderbook_df['minute'])
    
    for idx, trade in oneinch_df.iterrows():
        trade_minute = trade['minute']
        trade_details = get_eth_trade_size(trade)
        eth_amount = trade_details['eth_amount']
        
        # Skip if out of orderbook time or missing orderbook for this minute
        if trade_minute < ob_min_time or trade_minute > ob_max_time:
            continue
        if trade_minute not in ob_times:
            continue
        if eth_amount < SMALL_TRADE_THRESHOLD:
            continue
            
        try:
            ob_data = orderbook_df[orderbook_df['minute'] == trade_minute]
            if ob_data.empty:
                continue
                
            hedge_pnl, hedge_price = calculate_hedging_pnl(trade, ob_data)
            warehouse_pnl, warehouse_price, _, _ = calculate_warehousing_pnl(trade, ohlcv_df, LOOKFORWARD_WINDOW)
            
            # Calc Corrs
            price_corr, volume_corr = calculate_correlations(trade, oneinch_df, ohlcv_df, CORRELATION_WINDOW)
            
            recommendation = "Warehouse" if warehouse_pnl > hedge_pnl else "Hedge"
            
            results.append({
                'trade_time': trade['block_time'],
                'trade_type': trade['trade_type'],
                'eth_amount': round(eth_amount, 4),
                'usdt_value': round(trade_details['usdt_amount'], 4),
                '1inch_price': round(trade_details['price_per_eth'], 4),
                'hedge_price': round(hedge_price, 4),
                'hedge_pnl': round(hedge_pnl, 4),
                'warehouse_price': round(warehouse_price, 4),
                'warehouse_pnl': round(warehouse_pnl, 4),
                'price_corr': price_corr,  
                'volume_corr': volume_corr,  
                'recommendation': recommendation,
                'tx_hash': trade['tx_hash'],
                'pool_name': trade.get('pool_name', 'unknown')})
       
        except Exception as e:
            print(f"Error processing trade {trade.get('tx_hash', 'unknown')}: {str(e)}")
            continue
    
    return pd.DataFrame(results)

# ====================== MAIN EXECUTION ======================
if __name__ == "__main__":
    print("Loading preprocessed data...")
    oneinch_df = pd.read_pickle('processed_data/oneinch_df.pkl')
    ohlcv_df = pd.read_pickle('processed_data/ohlcv_df.pkl')
    orderbook_df = pd.read_pickle('processed_data/orderbook_df.pkl')
    
    # Convert timestamps
    oneinch_df['block_time'] = pd.to_datetime(oneinch_df['block_time'])
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'])
    orderbook_df['timestamp'] = pd.to_datetime(orderbook_df['timestamp'])
    
    # Create minute columns
    oneinch_df['minute'] = oneinch_df['block_time'].dt.floor('min')
    ohlcv_df['minute'] = ohlcv_df['timestamp'].dt.floor('min')
    orderbook_df['minute'] = orderbook_df['timestamp'].dt.floor('min')

    # Run analysis
    print("\nRunning flow internalization analysis...")
    results_df = flow_internalization_model(oneinch_df, ohlcv_df, orderbook_df)

    # Sort by trade_time (earliest first)
    results_df = results_df.sort_values('trade_time')

    # Ensure consistent formatting in final output
    float_cols = results_df.select_dtypes(include=[np.float64]).columns
    results_df[float_cols] = results_df[float_cols].round(4)

    # Save results
    os.makedirs('results', exist_ok=True)
    results_df.to_csv('results/flow_internalization_results.csv', index=False, float_format='%.4f')
 
    print("\n=== Analysis Results ===")
    print(f"Total trades analyzed: {len(results_df)}")
    
    if not results_df.empty:
        print("\nStrategy Performance Summary:")
        print(results_df[['hedge_pnl', 'warehouse_pnl']].describe())
        
        print("\nCorr Analysis:")
        print(f"Average Px Corr: {results_df['price_corr'].mean():.2f}")
        print(f"Average Volume Corr: {results_df['volume_corr'].mean():.2f}")
        
        print("\nRecommendation Distribution:")
        print(results_df['recommendation'].value_counts())
        
        # Plot trade sizes in ETH
        plt.figure(figsize=(12, 6))
        results_df['eth_amount'].hist(bins=50)
        plt.title('Processed Trade Sizes (ETH)')
        plt.xlabel('ETH Amount')
        plt.ylabel('# Trades')
        plt.xlim(0, 60)  # This sets x-axis range from 0 to 60
        plt.savefig('results/processed_trade_sizes.png')
        plt.close()
   
        print("\nVisualizations saved to 'results' directory")    
    print("\nAnalysis complete!")

Loading preprocessed data...

Running flow internalization analysis...

=== Analysis Results ===
Total trades analyzed: 1017

Strategy Performance Summary:
           hedge_pnl  warehouse_pnl
count    1017.000000    1017.000000
mean     -382.344708       3.509384
std      5911.043910     468.929301
min   -143999.379200   -7117.269100
25%        -0.782800      -0.983600
50%        -0.039900       0.038600
75%         0.020700       1.453900
max        12.965700   11753.995600

Corr Analysis:
Average Px Corr: 0.28
Average Volume Corr: -0.08

Recommendation Distribution:
recommendation
Warehouse    570
Hedge        447
Name: count, dtype: int64

Visualizations saved to 'results' directory

Analysis complete!


In [21]:
# Step2b-ii Slippage Parameter - Calibration
# Calibrate slippage formula with actual Orderbook liquidity
'''
Script:
Loads your saved 1inch, bids, asks DF (from your Step 2a outputs).
For each trade (min size specified below), finds corresp. orderbook at the trade's minute.
Compares formulaic slippage model to actual Binance ETH/USD Orderbook VWAP for req'd trade size.
Outputs a CSV with both slippages for comparison
Even though Step2a checks for order book existence at minute level, actual order book might not hv enough depth for large trades
Script checks vwap_fill_completed which fails when order book can't fully absorb trade size
'''
import pandas as pd
import numpy as np
import os
from datetime import datetime

# --- Config ---
MIN_TRADE_SIZE_ETH = 1.0  # Minimum ETH trade size to analyze
SLIPPAGE_MODEL = lambda x: 0.0002 * x  # eg. 0.0002 =  0.02% per ETH

def get_eth_trade_size(trade):
    trade_type = str(trade['trade_type']).upper()
    if trade_type in ['WETH_SELL', 'SELL']:
        return trade['sold_amount']
    else:  # WETH_BUY or BUY
        return trade['bought_amount']

def calculate_formulaic_slippage_price(side, top_px, eth_amount, slippage_model):
    slippage = slippage_model(eth_amount)
    if side == "sell":
        executed_price = top_px * (1 - slippage)
        slippage_pct = slippage * 100  # Convert to %
    else:
        executed_price = top_px * (1 + slippage)
        slippage_pct = slippage * 100  # Convert to %
    return executed_price, slippage_pct

def calculate_orderbook_vwap(orderbook_levels, eth_amount):
    """
    Calc VWAP & slippage using actual orderbook levels.
    orderbook_levels:    pd.DataFrame with columns ['price', 'amount'] sorted best-to-worst
    eth_amount:          amount to fill
    Returns:             vwap_price, slippage_percent, total_filled
    """
    amount_remaining = eth_amount
    total_cost = 0.0
    total_filled = 0.0

    for idx, row in orderbook_levels.iterrows():
        px = float(row['price'])
        size = float(row['amount'])
        take = min(size, amount_remaining)
        total_cost += px * take
        total_filled += take
        amount_remaining -= take
        if amount_remaining <= 1e-8:
            break

    if total_filled == 0:
        return float('nan'), float('nan'), 0

    vwap = total_cost / total_filled
    top_px = float(orderbook_levels.iloc[0]['price'])
    actual_slippage_pct = abs(vwap - top_px) / top_px * 100
    return vwap, actual_slippage_pct, total_filled

def compare_slippage(trade_row, bids_df, asks_df):
    eth_amount = get_eth_trade_size(trade_row)
    trade_type = trade_row['trade_type']
    orderbook_minute = trade_row['minute']
    block_time = trade_row.get('block_time', pd.NaT)

    if str(trade_type).upper() in ['WETH_SELL', 'SELL']:
        side = "sell"
        relevant_book = bids_df[bids_df['minute'] == orderbook_minute].sort_values('price', ascending=False)
    else:
        side = "buy"
        relevant_book = asks_df[asks_df['minute'] == orderbook_minute].sort_values('price', ascending=True)

    if relevant_book.empty:
        return {'block_time': block_time, 'orderbook_minute': orderbook_minute, 'trade_type': trade_type,
            'eth_amount': eth_amount, 'orderbook_found': False}

    top_px = float(relevant_book.iloc[0]['price'])

    # Formulaic slippage
    formulaic_px, formulaic_slip_pct = calculate_formulaic_slippage_price(side, top_px, eth_amount, SLIPPAGE_MODEL)

    # Actual orderbook VWAP slippage
    vwap_px, actual_slip_pct, total_filled = calculate_orderbook_vwap(relevant_book[['price', 'amount']], eth_amount)
    vwap_fill_completed = total_filled >= eth_amount * 0.999  # Allow 0.1% tolerance

    return {
        'block_time': block_time,
        'orderbook_minute': orderbook_minute,
        'trade_type': trade_type,
        'eth_amount': eth_amount,
        'orderbook_found': True,
        'top_px': top_px,
        'formulaic_px': formulaic_px,
        'formulaic_slippage_pct': formulaic_slip_pct,
        'orderbook_vwap_px': vwap_px,
        'orderbook_actual_slippage_pct': actual_slip_pct,
        'vwap_fill_completed': vwap_fill_completed,
        'vwap_total_filled': total_filled}

def format_results_df(df):
    """Apply specific decimal formatting to different column types"""
    df_formatted = df.copy()
    
    # Price columns - 2 decimals (force 2 decimal places)
    price_cols = ['top_px', 'formulaic_px', 'orderbook_vwap_px']
    for col in price_cols:
        if col in df_formatted.columns:
            df_formatted[col] = df_formatted[col].apply(
                lambda x: f"{float(x):.2f}" if pd.notna(x) else "")
    
    # % columns - 3 decimals
    pct_cols = ['formulaic_slippage_pct', 'orderbook_actual_slippage_pct']
    for col in pct_cols:
        if col in df_formatted.columns:
            df_formatted[col] = df_formatted[col].apply(
                lambda x: f"{x:.3f}" if pd.notna(x) else "")
    
    # Amount columns - 2 decimals
    amount_cols = ['eth_amount', 'vwap_total_filled']
    for col in amount_cols:
        if col in df_formatted.columns:
            df_formatted[col] = df_formatted[col].apply(
                lambda x: f"{x:.2f}" if pd.notna(x) else "")
    
    return df_formatted

if __name__ == "__main__":
    print(f"Starting slippage calibration at {datetime.now().isoformat()}")
    
    # Load processed data
    print("Loading data files...")
    oneinch_df = pd.read_pickle("processed_data/oneinch_df.pkl")
    orderbook_df = pd.read_pickle("processed_data/orderbook_df.pkl")

    # Process orderbook data
    bids_df = orderbook_df[orderbook_df['type'] == 'bids'].copy()
    asks_df = orderbook_df[orderbook_df['type'] == 'asks'].copy()
    
    # Convert to numeric types
    for df in [bids_df, asks_df]:
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
    
    # Process trades
    print(f"Processing {len(oneinch_df)} trades...")
    results = []
    for idx, row in oneinch_df.iterrows():
        eth_amount = get_eth_trade_size(row)
        if eth_amount < MIN_TRADE_SIZE_ETH:
            continue
            
        try:
            comp = compare_slippage(row, bids_df, asks_df)
            results.append(comp)
        except Exception as e:
            print(f"Error processing trade {idx}: {str(e)}")
            continue

    # Create and format results DF
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('block_time', ascending=True)
    
    # Reorder columns
    col_order = ['block_time', 'orderbook_minute', 'trade_type', 'eth_amount', 'orderbook_found', 'top_px', 'formulaic_px', 'orderbook_vwap_px',
        'formulaic_slippage_pct', 'orderbook_actual_slippage_pct', 'vwap_fill_completed', 'vwap_total_filled']
    results_df = results_df[[c for c in col_order if c in results_df.columns]]
    
    # Format numbers
    formatted_df = format_results_df(results_df)
    
    # Save results
    os.makedirs('results', exist_ok=True)
    output_path = 'results/slippage_comparison_vs_orderbook.csv'
    formatted_df.to_csv(output_path, index=False)
    
    print(f"\nAnalysis completed at {datetime.now().isoformat()}")
    print(f"Results saved to {output_path}")
    print(f"Total trades analyzed: {len(results_df)}")
    print(f"Breakdown:")
    print(f"- Trades with orderbook data: {results_df['orderbook_found'].sum()}")
    print(f"- Trades fully filled: {results_df['vwap_fill_completed'].sum()}")
    
    # Show sample output
    print("\nSample output (first 3 rows):")
    print(formatted_df.head(3).to_string())

Starting slippage calibration at 2025-05-22T21:47:06.000974
Loading data files...
Processing 1795 trades...

Analysis completed at 2025-05-22T21:47:06.798730
Results saved to results/slippage_comparison_vs_orderbook.csv
Total trades analyzed: 445
Breakdown:
- Trades with orderbook data: 316
- Trades fully filled: 316

Sample output (first 3 rows):
                   block_time          orderbook_minute trade_type eth_amount  orderbook_found   top_px formulaic_px orderbook_vwap_px formulaic_slippage_pct orderbook_actual_slippage_pct vwap_fill_completed vwap_total_filled
444 2025-05-05 08:17:47+00:00 2025-05-05 08:17:00+00:00  WETH_SELL       2.20             True  1825.41      1824.61           1825.41                  0.044                         0.000                True              2.20
443 2025-05-05 08:40:35+00:00 2025-05-05 08:40:00+00:00  WETH_SELL      20.04             True  1828.71      1821.38           1828.66                  0.401                         0.003           

In [22]:
'''
Step2c. Flow Internalization Model - Inventory Aware
'''
import pandas as pd
import os

def track_portfolio(flow_internalization_df):
    """
    Tracks portfolio value and PnL based on flow internalization decisions
    Args:      flow_internalization_df: DF w/trade details & recommendations
        
    Returns:   results_df: DataFrame with portfolio tracking results
               initial_price: Starting ETH price
               initial_portfolio_value: Starting portfolio value
    """
    results = []
    
    # Initialize Balances
    eth_balance = 100.0             # Start w/ ETH 100
    usdt_balance = 100000.0         # Start w/ USDT 100,000
    
    # Get Initial Px & calc Initial Portfolio Value
    initial_price = flow_internalization_df.iloc[0]['1inch_price']
    initial_portfolio_value = eth_balance * initial_price + usdt_balance
    
    for idx, trade in flow_internalization_df.iterrows():
        try:
            # Get trade details
            eth_amount = trade['eth_amount']
            usdt_value = trade['usdt_value']
            current_price = trade['1inch_price']
            recommendation = trade['recommendation']
            trade_type = trade['trade_type']
            hedge_pnl = trade.get('hedge_pnl', 0)  # Default to 0 if column missing
            warehouse_pnl = trade.get('warehouse_pnl', 0)
            
            # For Hedge trades (offset on Binance)
            if recommendation == 'Hedge':
                # ETH balance remains unchanged (pass-through)
                # USDT balance only changes by hedge PnL
                usdt_balance += hedge_pnl
            
            # For Warehouse trades (inventory changes)
            elif recommendation == 'Warehouse':
                if trade_type == 'WETH_SELL':
                    # Client sells WETH to us - we receive WETH and give USDT
                    eth_balance += eth_amount
                    usdt_balance -= usdt_value
                    # Apply warehouse PnL
                    usdt_balance += warehouse_pnl
                else:  # WETH_BUY
                    # Client buys WETH from us - we give WETH and receive USDT
                    eth_balance -= eth_amount
                    usdt_balance += usdt_value
                    # Apply warehouse PnL
                    usdt_balance += warehouse_pnl
            
            # Calc current portfolio value
            portfolio_value = eth_balance * current_price + usdt_balance
            cum_pnl = portfolio_value - initial_portfolio_value
            
            # Record results
            results.append({
                'trade_time': trade['trade_time'],
                'trade_type': trade_type,
                'eth_amount': round(eth_amount, 4),
                'usdt_value': round(usdt_value, 4),
                '1inch_price': round(current_price, 4),
                'recommendation': recommendation,
                'hedge_pnl': round(hedge_pnl, 4) if recommendation == 'Hedge' else 0,
                'warehouse_pnl': round(warehouse_pnl, 4) if recommendation == 'Warehouse' else 0,
                'ETH_Net_Cum_Balance': round(eth_balance, 2),
                'USDT_Cum_Balance': round(usdt_balance, 2),
                'Portfolio_Value': round(portfolio_value, 2),
                'Cum_PnL': round(cum_pnl, 2)})
            
        except Exception as e:
            print(f"Error processing trade at {trade['trade_time']}: {str(e)}")
            continue
    
    return pd.DataFrame(results), initial_price, initial_portfolio_value

def main():
    print("Loading flow internalization data...")
    
    # Load and prepare data
    flow_internalization_df = pd.read_csv('results/flow_internalization_results.csv')
    
    # Convert numeric columns
    numeric_cols = ['eth_amount', 'usdt_value', '1inch_price', 'hedge_pnl', 'warehouse_pnl']
    flow_internalization_df[numeric_cols] = flow_internalization_df[numeric_cols].apply(
        pd.to_numeric, errors='coerce')
    
    # Standardize trade_type values
    flow_internalization_df['trade_type'] = flow_internalization_df['trade_type'].replace({
        'WETH_SELL': 'WETH_SELL', 'WETH_BUY': 'WETH_BUY'})
    
    # Sort chronologically
    flow_internalization_df = flow_internalization_df.sort_values('trade_time')
    
    # Run portfolio tracker
    results_df, initial_price, initial_value = track_portfolio(flow_internalization_df)
    
    # Save results
    os.makedirs('results', exist_ok=True)
    results_df.to_csv('results/portfolio_tracking_results.csv', index=False, float_format='%.2f')
    
    # Print summary
    print("\n=== Initial Values ===")
    print(f"INITIAL_ETH: 100.00")
    print(f"1inch_price: {initial_price:,.4f}")
    print(f"INITIAL_USDT: 100,000.00")
    print(f"Initial Portfolio Value (USDT): {initial_value:,.2f}")
    
    print("\n=== First 5 Trades ===")
    print(results_df.head().to_string(float_format=lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else str(x)))
    
    print("\n=== Last 5 Trades ===")
    print(results_df.tail().to_string(float_format=lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else str(x)))

if __name__ == "__main__":
    main()

Loading flow internalization data...

=== Initial Values ===
INITIAL_ETH: 100.00
1inch_price: 1,832.9097
INITIAL_USDT: 100,000.00
Initial Portfolio Value (USDT): 283,290.97

=== First 5 Trades ===
                  trade_time trade_type  eth_amount  usdt_value  1inch_price recommendation  hedge_pnl  warehouse_pnl  ETH_Net_Cum_Balance  USDT_Cum_Balance  Portfolio_Value    Cum_PnL
0  2025-05-05 08:04:23+00:00  WETH_SELL        0.03       49.49     1,832.91      Warehouse       0.00          -0.16               100.03         99,950.35       283,290.81      -0.16
1  2025-05-05 08:10:11+00:00   WETH_BUY        0.02       30.00     1,699.44          Hedge      -2.23           0.00               100.03         99,948.12       269,938.13 -13,352.84
2  2025-05-05 08:11:23+00:00  WETH_SELL        0.06      109.50     1,824.94      Warehouse       0.00           0.08               100.09         99,838.70       282,491.43    -799.54
3  2025-05-05 08:15:47+00:00  WETH_SELL        0.01       22.71

In [28]:
print(results.columns.tolist())

['trade_time', 'trade_type', 'eth_amount', 'usdt_value', '1inch_price', 'recommendation', 'hedge_pnl', 'warehouse_pnl', 'ETH_Net_Cum_Balance', 'USDT_Cum_Balance', 'Portfolio_Value', 'Cum_PnL']


In [23]:
# At end of Step2d - load up the .html generated file & watch the portfolio evolve thru time.
#Step2d_i Visualization Setup start
import pandas as pd
import numpy as np

def prepare_inventory_data(results_df):
    # Prepare & downsample Inventory data for visualization
    # Convert 'trade_time' to datetime if not already
    results_df['trade_time'] = pd.to_datetime(results_df['trade_time'])
    
    # Select only numeric columns for aggregation
    numeric_columns = results_df.select_dtypes(include=[np.number]).columns
    results_df_numeric = results_df[['trade_time'] + list(numeric_columns)]
    
    # Downsample data to 1 frame per minute
    results_df_downsampled = results_df_numeric.resample('min', on='trade_time').mean().reset_index()
    
    return results_df_downsampled

In [24]:
#Step 2d_ii
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_static_inventory_plot(results_df_downsampled):
    """Create a static inventory plot."""
    # Create figure with subplots
    fig = make_subplots(rows=2, cols=1, subplot_titles=('ETH Inventory Evolution', 'USDT Inventory Evolution'), vertical_spacing=0.15)
    
    # Add ETH inventory traces
    fig.add_trace(go.Scatter(x=results_df_downsampled['trade_time'], 
                   y=results_df_downsampled['hedge_eth_balance'], name='Hedge ETH', line=dict(color='blue')), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=results_df_downsampled['trade_time'], 
                   y=results_df_downsampled['warehouse_eth_balance'], name='Warehouse ETH', line=dict(color='orange')), row=1, col=1)
    
    # Add USDT inventory traces
    fig.add_trace(go.Scatter(x=results_df_downsampled['trade_time'],  
                    y=results_df_downsampled['hedge_usdt_balance'], name='Hedge USDT', line=dict(color='blue'), showlegend=False), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=results_df_downsampled['trade_time'], 
                    y=results_df_downsampled['warehouse_usdt_balance'], name='Warehouse USDT', line=dict(color='orange'), showlegend=False), row=2, col=1)
    
    # Update layout
    fig.update_layout(height=800, title_text="Inventory Evolution with Trade Flow", hovermode="x unified")
    
    # Update y-axis titles
    fig.update_yaxes(title_text="ETH Balance", row=1, col=1)
    fig.update_yaxes(title_text="USDT Balance", row=2, col=1)
    
    return fig

In [25]:
#Step 2d_iii
def add_animation_frames(fig, results_df_downsampled):
    """Add animation frames to inventory plot."""
    frames = []
    for i in range(1, len(results_df_downsampled) + 1):
        frame_df = results_df_downsampled.iloc[:i]
        frames.append(go.Frame(data=[go.Scatter(x=frame_df['trade_time'], y=frame_df['hedge_eth_balance']),        # ETH Hedge
                go.Scatter(x=frame_df['trade_time'], y=frame_df['warehouse_eth_balance']),                         # ETH Warehouse
                go.Scatter(x=frame_df['trade_time'], y=frame_df['hedge_usdt_balance']),                            # USDT Hedge
                go.Scatter(x=frame_df['trade_time'], y=frame_df['warehouse_usdt_balance']),],                      # USDT Warehouse
            name=f'frame_{i}'))
    
    fig.frames = frames
    
    # Add play/pause buttons
    fig.update_layout(updatemenus=[{"type": "buttons", "buttons": [{"label": "Play", "method": "animate",
                    "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
                {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}], "x": 0.1, "y": 0,}])

    return fig

In [34]:
import pandas as pd
import plotly.graph_objects as go
from tqdm import tqdm

# Load results DF
results = pd.read_csv('results/portfolio_tracking_results.csv')

# Prep & downsample the data
results['trade_time'] = pd.to_datetime(results['trade_time'])
numeric_columns = results.select_dtypes(include=[float, int]).columns
results_numeric = results[['trade_time'] + list(numeric_columns)]
results_downsampled = results_numeric.resample('5min', on='trade_time').mean().reset_index()

# Scale down USDT balance by 10x
results_downsampled['USDT_Cum_Balance_scaled'] = results_downsampled['USDT_Cum_Balance'] / 10

# Create figure with secondary y-axis
fig = go.Figure()

# Add ETH Net Cumulative Balance to primary y-axis (left)
fig.add_trace(go.Scatter(
    x=results_downsampled['trade_time'],
    y=results_downsampled['ETH_Net_Cum_Balance'],
    name='ETH Net Cumulative Balance',
    line=dict(color='blue')))

# Add the other traces to secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=results_downsampled['trade_time'],
    y=results_downsampled['Portfolio_Value'],
    name='Portfolio Value',
    line=dict(color='green'),
    yaxis='y2'))

fig.add_trace(go.Scatter(
    x=results_downsampled['trade_time'],
    y=results_downsampled['USDT_Cum_Balance_scaled'],
    name='USDT Cumulative Balance (scaled 1/10)',
    line=dict(color='orange'),
    yaxis='y2'))

fig.add_trace(go.Scatter(
    x=results_downsampled['trade_time'],
    y=results_downsampled['Cum_PnL'],
    name='Cumulative PnL',
    line=dict(color='purple'),
    yaxis='y2'))

# Add annotation about scaling
fig.add_annotation(
    x=0.02,
    y=0.95,
    xref='paper',
    yref='paper',
    text="<b>Note:</b> USDT Balance values are scaled down by 10x",
    showarrow=False,
    font=dict(size=12),
    bgcolor="white",
    bordercolor="black",
    borderwidth=1)

# Create axis objects
fig.update_layout(
    title='Portfolio Tracking Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(
        title='ETH Net Cumulative Balance',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')),
    yaxis2=dict(
        title='Portfolio Value, USDT Balance (scaled), & PnL',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        anchor='x',
        overlaying='y',
        side='right'),
    legend=dict(x=1.1, y=1.0),
    margin=dict(l=100, r=100, t=80, b=80)  # Add margin for annotation)

# Add animation frames with progress bar
frames = []
print("Generating animation frames...")
for i in tqdm(range(1, len(results_downsampled) + 1), desc="Frames Progress"):
    frame_df = results_downsampled.iloc[:i]
    frames.append(go.Frame(
        data=[
            go.Scatter(x=frame_df['trade_time'], y=frame_df['ETH_Net_Cum_Balance']),
            go.Scatter(x=frame_df['trade_time'], y=frame_df['Portfolio_Value'], yaxis='y2'),
            go.Scatter(x=frame_df['trade_time'], y=frame_df['USDT_Cum_Balance_scaled'], yaxis='y2'),
            go.Scatter(x=frame_df['trade_time'], y=frame_df['Cum_PnL'], yaxis='y2')],
        name=f'frame_{i}'))

fig.frames = frames

# Add play/pause buttons
fig.update_layout(
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]},
            {
                "label": "Pause",
                "method": "animate",
                "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}]}])

# Save as HTML file
fig.write_html("Portfolio-Tracking-Visualization.html")
print("Visualization saved to Portfolio-Tracking-Visualization.html")

Generating animation frames...


Frames Progress: 100%|███████████████████████| 590/590 [00:01<00:00, 366.39it/s]


Visualization saved to Portfolio-Tracking-Visualization.html
